In [4]:
pip install pyyaml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\Sreeja S\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import os
import yaml
import pandas as pd
from collections import defaultdict

In [5]:

# Define input and output paths
input_root = r'D:\00Stocks\data'.replace('\x00', '')  # path
output_dir = 'csv_output'
os.makedirs(output_dir, exist_ok=True)

# Dictionary to collect data for each symbol
symbol_data = defaultdict(list)

# Loop through each month's folder

for month in os.listdir(input_root):
    month_path = os.path.join(input_root, month)
    if not os.path.isdir(month_path):
        continue  

    # Loop through each YAML file in the month folder
    for filename in os.listdir(month_path):
        if filename.endswith('.yaml') or filename.endswith('.yml'):
            file_path = os.path.join(month_path, filename)

            with open(file_path, 'r') as f:
                try:
                    yaml_data = yaml.safe_load(f)
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
                    continue

            # Extracting date from filename
            date = filename.replace('.yaml', '').replace('.yml', '')

            # Loop through each symbol in the YAML file
            for event in yaml_data:  # ✅ yaml_data is a list of dicts
                symbol = event.get('category', 'unknown') 
                record = {
                'date': date,
                 **event
                 }
                symbol_data[symbol].append(record)

                if not isinstance(event, dict):
                    continue  # Skip malformed entries
                    record = {'date': date}
                record.update(event)  # Merge symbol data into record
                symbol_data[symbol].append(record)

                custom_names = {
    'maintenance': 'Server_Maintenance_Log.csv',
    'update': 'Security_Updates.csv',
    'upgrade': 'System_Upgrades.csv',
    'audit': 'Audit_Logs.csv',
    'reconfiguration': 'Network_Reconfig.csv',
    'unknown': 'Uncategorized.csv'
}

# Looping to write each symbol's data to its own CSV file
for symbol, records in symbol_data.items():
    df = pd.DataFrame(records)
    df.sort_values(by='date', inplace=True)
    csv_name = custom_names.get(symbol, f"{symbol}.csv")
    df.to_csv(os.path.join(output_dir, csv_name), index=False)
    print(f"Saved: {csv_name}")



Saved: Uncategorized.csv


In [6]:
df

,date,Ticker,close,high,low,month,open,volume
0,2023-10-03 05:30:00,SBIN,602.95,604.90,589.60,2023-10,596.60,15322196
72,2023-10-03 05:30:00,SUNPHARMA,1141.45,1161.95,1139.25,2023-10,1159.55,1946966
71,2023-10-03 05:30:00,DRREDDY,1092.34,1117.38,1090.00,2023-10,1117.38,2065240
70,2023-10-03 05:30:00,DRREDDY,1092.34,1117.38,1090.00,2023-10,1117.38,2065240
69,2023-10-03 05:30:00,HDFCLIFE,633.55,637.50,628.55,2023-10,634.45,3454441
...,...,...,...,...,...,...,...,...
28327,2024-11-22 05:30:00,TRENT,6652.80,6678.00,6430.00,2024-11,6459.90,833104
28326,2024-11-22 05:30:00,TRENT,6652.80,6678.00,6430.00,2024-11,6459.90,833104
28325,2024-11-22 05:30:00,INFY,1902.25,1914.05,1834.00,2024-11,1854.10,6301486
28335,2024-11-22 05:30:00,TECHM,1747.45,1749.85,1702.25,2024-11,1714.75,2344087


In [7]:
# Cleaning process

df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date

df['Ticker'] = df['Ticker'].str.strip()

In [8]:
df

,date,Ticker,close,high,low,month,open,volume
0,2023-10-03,SBIN,602.95,604.90,589.60,2023-10,596.60,15322196
72,2023-10-03,SUNPHARMA,1141.45,1161.95,1139.25,2023-10,1159.55,1946966
71,2023-10-03,DRREDDY,1092.34,1117.38,1090.00,2023-10,1117.38,2065240
70,2023-10-03,DRREDDY,1092.34,1117.38,1090.00,2023-10,1117.38,2065240
69,2023-10-03,HDFCLIFE,633.55,637.50,628.55,2023-10,634.45,3454441
...,...,...,...,...,...,...,...,...
28327,2024-11-22,TRENT,6652.80,6678.00,6430.00,2024-11,6459.90,833104
28326,2024-11-22,TRENT,6652.80,6678.00,6430.00,2024-11,6459.90,833104
28325,2024-11-22,INFY,1902.25,1914.05,1834.00,2024-11,1854.10,6301486
28335,2024-11-22,TECHM,1747.45,1749.85,1702.25,2024-11,1714.75,2344087


In [9]:

# Creating output folder
output_dir = 'split_by_Ticker'
os.makedirs(output_dir, exist_ok=True)

# Group by category and saving each to a separate file
for Ticker, group_df in df.groupby('Ticker'):
    filename = f"{Ticker.replace(' ', '_')}.csv"
    output_path = os.path.join(output_dir, filename)
    group_df.to_csv(output_path, index=False)
    print(f"Saved: {output_path}")

Saved: split_by_Ticker\ADANIENT.csv
Saved: split_by_Ticker\ADANIPORTS.csv
Saved: split_by_Ticker\APOLLOHOSP.csv
Saved: split_by_Ticker\ASIANPAINT.csv
Saved: split_by_Ticker\AXISBANK.csv
Saved: split_by_Ticker\BAJAJ-AUTO.csv
Saved: split_by_Ticker\BAJAJFINSV.csv
Saved: split_by_Ticker\BAJFINANCE.csv
Saved: split_by_Ticker\BEL.csv
Saved: split_by_Ticker\BHARTIARTL.csv
Saved: split_by_Ticker\BPCL.csv
Saved: split_by_Ticker\BRITANNIA.csv
Saved: split_by_Ticker\CIPLA.csv
Saved: split_by_Ticker\COALINDIA.csv
Saved: split_by_Ticker\DRREDDY.csv
Saved: split_by_Ticker\EICHERMOT.csv
Saved: split_by_Ticker\GRASIM.csv
Saved: split_by_Ticker\HCLTECH.csv
Saved: split_by_Ticker\HDFCBANK.csv
Saved: split_by_Ticker\HDFCLIFE.csv
Saved: split_by_Ticker\HEROMOTOCO.csv
Saved: split_by_Ticker\HINDALCO.csv
Saved: split_by_Ticker\HINDUNILVR.csv
Saved: split_by_Ticker\ICICIBANK.csv
Saved: split_by_Ticker\INDUSINDBK.csv
Saved: split_by_Ticker\INFY.csv
Saved: split_by_Ticker\ITC.csv
Saved: split_by_Ticker\JSWSTE

In [10]:
from glob import glob

In [11]:

# Path to your extracted 50 CSV files
csv_folder = r'D:\00Stocks\split_by_Ticker' 

# Gather all CSV file paths
csv_files = glob(os.path.join(csv_folder, '*.csv'))

summary_list = []

for file in csv_files:
    symbol = os.path.basename(file).replace('.csv', '')
    df = pd.read_csv(file)

    # date sorting
    df = df.sort_values('date')

    # Get start and end prices
    start_price = df.iloc[0]['close']
    end_price = df.iloc[-1]['close']

    # Yearly return calculation
    yearly_return = ((end_price - start_price) / start_price) * 100

    # Average price and volume
    avg_price = df['close'].mean()
    avg_volume = df['volume'].mean()

    summary_list.append({
        'symbol': symbol,
        'yearly_return': yearly_return,
        'avg_price': avg_price,
        'avg_volume': avg_volume
    })

# Create summary DataFrame
summary_df = pd.DataFrame(summary_list)

# Categorize stocks
summary_df['trend'] = summary_df['yearly_return'].apply(lambda x: 'green' if x > 0 else 'red')


# Top 10 Green Stocks
top_10_green = summary_df[summary_df['yearly_return'] > 0].sort_values(by='yearly_return', ascending=False).head(10)

# Top 10 Loss Stocks
top_10_loss = summary_df.sort_values(by='yearly_return').head(10)


# Market Summary
trend_counts = summary_df['trend'].value_counts()
green_count = (summary_df['trend'] == 'Green').sum()
red_count = (summary_df['trend'] == 'Red').sum()
avg_price_all = summary_df['avg_price'].mean()
avg_volume_all = summary_df['avg_volume'].mean()

# ---------------- Output ----------------
print("\nTop 10 Green Stocks:\n", top_10_green[['symbol', 'yearly_return']])
print("\nTop 10 Loss Stocks:\n", top_10_loss[['symbol', 'yearly_return']])

print("\nMarket Summary:")

green_count = trend_counts.get('green', 0)
red_count = trend_counts.get('red', 0)

print(f"Green: {green_count}, Red: {red_count}")
print(f"Average Price Across All Stocks: {avg_price_all:.2f}")
print(f"Average Volume Across All Stocks: {avg_volume_all:.0f}")



Top 10 Green Stocks:
         symbol  yearly_return
47       TRENT     223.092613
8          BEL     101.760057
30         M&M      95.976974
5   BAJAJ-AUTO      89.011153
9   BHARTIARTL      69.599049
35   POWERGRID      68.854924
10        BPCL      67.477150
20  HEROMOTOCO      58.976655
40   SUNPHARMA      57.282404
17     HCLTECH      53.257447

Top 10 Loss Stocks:
         symbol  yearly_return
24  INDUSINDBK     -30.458409
3   ASIANPAINT     -21.935046
7   BAJFINANCE     -16.110874
0     ADANIENT      -6.670856
22  HINDUNILVR      -0.957916
32   NESTLEIND       0.707141
28   KOTAKBANK       1.991836
6   BAJAJFINSV       2.549566
46       TITAN       3.518185
31      MARUTI       6.926712

Market Summary:
Green: 45, Red: 5
Average Price Across All Stocks: 2449.42
Average Volume Across All Stocks: 6833475


In [12]:
pip install sqlalchemy pymysql pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\Sreeja S\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
from sqlalchemy import create_engine

# ---------------------- CONFIG ----------------------
# MySQL connection settings
db_user = 'root'
db_password = '5455'
db_host = 'localhost'
db_port = 3306
db_name = 'Stock_analysis_2024'

# Folder paths
csv_folder_path = r'D:\00Stocks\split_by_Ticker'  # folder with 50 CSVs
common_csv_path = r'D:\00Stocks\Sector_data_Sheet1.csv'  # single CSV file
# -----------------------------------------------------

# Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Function to upload a CSV to MySQL
def upload_csv_to_mysql(file_path, table_name):
    try:
        df = pd.read_csv(file_path)
        df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
        print(f"✅ Uploaded: {file_path} -> Table: {table_name}")
    except Exception as e:
        print(f"❌ Failed to upload {file_path}: {e}")

# Upload 50 CSVs
for filename in os.listdir(csv_folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_folder_path, filename)
        table_name = os.path.splitext(filename)[0]  # Use filename (without .csv) as table name
        upload_csv_to_mysql(file_path, table_name)

# Upload the common CSV file
common_table_name = os.path.splitext(os.path.basename(common_csv_path))[0]
upload_csv_to_mysql(common_csv_path, common_table_name)

print("\n🎉 All files processed successfully!")


C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'ADANIENT' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'ADANIPORTS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'APOLLOHOSP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine

✅ Uploaded: D:\00Stocks\split_by_Ticker\ADANIENT.csv -> Table: ADANIENT
✅ Uploaded: D:\00Stocks\split_by_Ticker\ADANIPORTS.csv -> Table: ADANIPORTS
✅ Uploaded: D:\00Stocks\split_by_Ticker\APOLLOHOSP.csv -> Table: APOLLOHOSP
✅ Uploaded: D:\00Stocks\split_by_Ticker\ASIANPAINT.csv -> Table: ASIANPAINT


C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'AXISBANK' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'BAJAJ-AUTO' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'BAJAJFINSV' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine

✅ Uploaded: D:\00Stocks\split_by_Ticker\AXISBANK.csv -> Table: AXISBANK
✅ Uploaded: D:\00Stocks\split_by_Ticker\BAJAJ-AUTO.csv -> Table: BAJAJ-AUTO
✅ Uploaded: D:\00Stocks\split_by_Ticker\BAJAJFINSV.csv -> Table: BAJAJFINSV
✅ Uploaded: D:\00Stocks\split_by_Ticker\BAJFINANCE.csv -> Table: BAJFINANCE


C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'BEL' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'BHARTIARTL' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'BPCL' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists

...

C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'SBIN' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'SHRIRAMFIN' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'SUNPHARMA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_

✅ Uploaded: D:\00Stocks\split_by_Ticker\SBIN.csv -> Table: SBIN
✅ Uploaded: D:\00Stocks\split_by_Ticker\SHRIRAMFIN.csv -> Table: SHRIRAMFIN
✅ Uploaded: D:\00Stocks\split_by_Ticker\SUNPHARMA.csv -> Table: SUNPHARMA
✅ Uploaded: D:\00Stocks\split_by_Ticker\TATACONSUM.csv -> Table: TATACONSUM
✅ Uploaded: D:\00Stocks\split_by_Ticker\TATAMOTORS.csv -> Table: TATAMOTORS


C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'TATASTEEL' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'TCS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'TECHM' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists

✅ Uploaded: D:\00Stocks\split_by_Ticker\TATASTEEL.csv -> Table: TATASTEEL
✅ Uploaded: D:\00Stocks\split_by_Ticker\TCS.csv -> Table: TCS
✅ Uploaded: D:\00Stocks\split_by_Ticker\TECHM.csv -> Table: TECHM
✅ Uploaded: D:\00Stocks\split_by_Ticker\TITAN.csv -> Table: TITAN
✅ Uploaded: D:\00Stocks\split_by_Ticker\TRENT.csv -> Table: TRENT
✅ Uploaded: D:\00Stocks\split_by_Ticker\ULTRACEMCO.csv -> Table: ULTRACEMCO
✅ Uploaded: D:\00Stocks\split_by_Ticker\WIPRO.csv -> Table: WIPRO
✅ Uploaded: D:\00Stocks\Sector_data_Sheet1.csv -> Table: Sector_data_Sheet1

🎉 All files processed successfully!


C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'ULTRACEMCO' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'WIPRO' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
C:\Users\Sreeja S\AppData\Local\Temp\ipykernel_19640\2504609774.py:23: UserWarning: The provided table name 'Sector_data_Sheet1' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=e